In [1]:
%load_ext autoreload
%autoreload 2

# 2. Projection Creation Example

This example shows how to create a projection and add coordinates to a dataset.

In [2]:
from rts.db.dao import DataAccessObject
from rts.db_settings import DATABASE_URL
from rts.db.queries import get_library_id_from_name, read_media_by_library_id, create_projection, update_projection, create_map_projection_feature
from rts.io.media import upload_media_files, create_square_atlases
from rts.api.models import Media, Projection, MapProjectionFeatureCreate
from rts.storage.storage import get_storage_client
import os
import random
from io import BytesIO
from PIL import Image
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
DataAccessObject().connect(DATABASE_URL)

ARCHIVE_BASE_PATH = os.getenv("BASE_PATH")

/Users/arattinger/Projects/rts/rts/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
archive_name =  "rts"
bucket_name =  archive_name
library_id =  get_library_id_from_name(archive_name)

In [4]:
# Retrieve media files for the projection and the atlas
media = read_media_by_library_id(library_id, 'image', 'thumbnail')
print(f"Found {len(media)} media files for library {library_id}")

Found 63 media files for library 1


In [5]:
# Retrieve and generate all the data we need for the projection and atlas
data = {
    'images': [],
    'coordinates': [],
}

for m in media:
    result = get_storage_client().download(bucket_name, m['media_path'])
    coordinates = [random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)]
    data['coordinates'].append(coordinates)
    data['images'].append(Image.open(BytesIO(result)))

In [6]:
# In this example we want to split the media files into multiple atlases

total_tiles = len(media)
atlas_width = 1024
max_tile_size = 128
max_tiles_per_atlas = (atlas_width // max_tile_size) ** 2
atlas_count = int(total_tiles / max_tiles_per_atlas) + 1

projection = Projection(
    projection_name="rts_random",
    version="0.0.1",
    library_id=library_id,
    model_name="random",
    model_params={},
    data={},
    dimension=3,
    atlas_folder_path="",
    atlas_width=atlas_width,
    tile_size=max_tile_size,
    atlas_count=atlas_count,
    total_tiles=total_tiles,
    tiles_per_atlas=max_tiles_per_atlas,
)
projection_id = create_projection(projection)['projection_id']

In [7]:
square_atlases = create_square_atlases("atlas_rts_001", projection_id, data['images'], width=atlas_width, max_tile_size=max_tile_size)

In [8]:
# for m in media:
count = 0
for atlas_order, key in enumerate(square_atlases):
    # print(len(square_atlases[key]['images']))
    for i in range(len(square_atlases[key]['images'])):
        create_map_projection_feature(
            MapProjectionFeatureCreate(
                projection_id=projection_id,
                media_id=media[count]['media_id'],
                atlas_order=atlas_order + 1,
                coordinates=[random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)],
                index_in_atlas=i,
            )
        )
        # print(i, count, atlas_order + 1)
        count += 1